# 1. ZH C csoport - Összesen 10 pont - 90 perc áll rendelkezésre a megoldáshoz

## Azonosító adatok

- NÉV: Urbán Eszter Klára
- NEPTUN KÓD: HI4SQE

## Feladatok

1. (2 pont) Adja meg az alábbi probléma jellemzőit, illetve cél és kezdő állapotát:
- Van egy speciális 64 különböző karakterből álló ABC.
- A kezdő állapotban bármilyen sorrendben lehetnek a karakterek. De minden karakter elhelyezkedik valamilyen pozicóban.
- A cél állapotban a karaktereknek a kezdő állapottal ellentétes irányban kell lenniük.

- Egy példa arra ha csak 5 darabból állna az ABC:
- Bemenet: A,B,C,D,E
- Kimenet: E,D,C,B,A
- A probléma ennek analógiája csak 64 karakterrel.

In [79]:
# Válasz helye. Válaszát szabályos python commentek között adja meg.
"""

Jellemzői:
H1 = {A}
H2 = {B}
H3 = {C}
.
.
.
H64 = {Z}

Kezdő állapot:
a0 = <A valamely sorrendje>

Cél állapot:
c = {<a1, a2, ..., a64> | <a1, a2, ..., a64> ∈ A ∧ reverse(a0)}

"""

'\n\nJellemzői:\nH1 = {A}\nH2 = {B}\nH3 = {C}\n.\n.\n.\nH64 = {Z}\n\nKezdő állapot:\na0 = <A valamely sorrendje>\n\nCél állapot:\nc = {<a1, a2, ..., a64> | <a1, a2, ..., a64> ∈ A ∧ reverse(a0)}\n\n'

### Futassa le az alábbi segéd osztályokat

In [80]:
class Problem:
    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        if isinstance(self.goal, list):
            for s in self.goal:
                if s==state:
                    return True

            return False
        else:
            return state == self.goal
        
    def path_cost(self, c, state1, action, state2):
        return c + 1

In [81]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(state = next_state, 
                         parent = self, 
                         action = action, 
                         path_cost = problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node

    def expand(self, problem):
        return [self.child_node(problem, action) for action in problem.actions(self.state)]

    def solution(self):
        return [node.action for node in self.path()[1:]]

    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

### Adott a következő állapottér reprezentáció

#### A vodka és a parasztok
- Egy vödörben 12 liter vodkát kell egyenlően elosztani két orosz paraszt között.
- Két 8 és 6 literes palackjuk van.

### Jellemzők

- A jellemzők legyenek a vödör és a két palack
- H1 = {0, 1, ..., 12}, vödör
- H2 = {0, 1, ..., 8}, palack 1
- H3 = {0, 1, ..., 6}, palack 2

### Állapotok halmaza

- A ⊆ H1xH2xH3
- Legyen a !-jel a negáció
- A = {<a1, a2, a3> | <a1, a2, a3> ∈ H1 x H2 x H3 ∧ a1+a2+a3=12}

### Kezdő állapot:

- a0 = <12, 0, 0>

### Célállapotok:

- C = <0, 6, 6>

### Operátorok:

- O = {oi,j} = {oi | i ∈ {1,2,3} ∧ j ∈ {1,2,3} ∧ i!=j} 
- Dom(oi)={<a1, a2, a3>  | <a1, a2, a3> ∈ A ∧ (a1>0 ∧ a2<max(H2)) v (a1>0 ∧ a3<max(H3) v (a2>0 ∧ a3<max(H3)) v (a2>0 ∧ a1<max(H1)) v (a3>0 ∧ a2<max(H2)) v (a3>0 ∧ a1<max(H1))}
- oi(<a1, a2, a3> = <b1, b2, b3>)
- m = min(ai, max(Hj) – aj)
- bk, ahol b=1,2,3
    - ak + m, ha k = j
    - ak - m, ha k = i
    - ak, egyébként

2. (3 pont) Készítsen egy osztályt, amely megvalósítja az állapotér reprezentációt a Problem és Node osztályok segítségével. Az állapottér reprezentáció megvalósításához származzon le a Problem osztályból és írja meg annak "állapot átmenet függvényét / operátorok / actions" és "operátor hatás definícióját / result". Ha szükséges bármilyen egyéb program csomag a futtatáshoz annak megadásáról se feledkezzen meg.

In [82]:
# Válasz helye. Válaszát futtatható python kóddal adja meg.

class Cup3(Problem):
    def actions(self, state):
        #operatorok definialasa
        acts = []
        twelve, eight, six = state
        if twelve > 0 and eight < 8:
            acts.append("12-->8")
        if twelve > 0 and six < 6:
            acts.append("12-->6")
        if eight > 0 and twelve < 12:
            acts.append("8-->12")
        if eight > 0 and six < 6:
            acts.append("8-->6")
        if six > 0 and twelve < 12:
            acts.append("6-->12")
        if six > 0 and eight < 8:
            acts.append("6-->8")
        return acts 

    def result(self, state, action):
        #ilyen irasa lehet zh
        #operátorok hatásának definiálása
        twelve, eight, six = state
        if action == "12-->8":
            m = min(twelve,8-eight)
            return(twelve-m,eight+m,six)
        if action == "12-->6":
            m = min(twelve,6-six)
            return(twelve-m,eight,six+m)
        if action == "8-->12":
            m = min(eight,12-twelve)
            return(twelve+m,eight-m,six)
        if action == "8-->6":
            m = min(eight,6-six)
            return(twelve,eight-m,six+m)
        if action == "6-->12":
            m = min(six,12-twelve)
            return(twelve+m,eight,six-m)
        if action == "6-->8":
            m = min(six,8-eight)
            return(twelve,eight+m,six-m)

3. (1 pont) Példányosítsa a reprezentációt a kezdő és cél állapottal majd írassa ki a kezdő sé cél állapotot a "print()" függvény segítségével 

In [83]:
# Válasz helye. Válaszát futtatható python kóddal adja meg.
c = Cup3((12, 0, 0), (0, 6, 6))
print(c.initial, c.goal)

(12, 0, 0) (0, 6, 6)


4. (1. pont) Írja meg a próba hiba módszert a megoldás megtalálásához.Írjon megjegyzés szabályos python formátumban amiben leírja, hogy pontosan mit csinál az algoritmus. Ha szükséges bármilyen egyéb program csomag a futtatáshoz annak megadásáról se feledkezzen meg.

In [84]:
# Válasz helye. Válaszát futtatható python kóddal adja meg.
import numpy as np

def trial_error(problem):
    # kezdo allapot
    state = Node(problem.initial)

    # végtelen ciklus definiálása
    while True:
        # ha a probléma megoldva akkor leállítjuk a végtelen ciklust
        if problem.goal_test(state.state):
            print('Got it')
            return state

        # a alkamazható operátorok segítségével
        # gyártsuk le az összes lehetséges utódot
        successors = state.expand(problem)

        # ha nincs új állapot(utód)
        if len(successors) == 0:
            return 'Unsolvable'

        # random választunk egy újat a legyártott utódok közül
        state = successors[np.random.randint(0, len(successors))]
        print(state.state)

5. (1 pont) Futtasa a próba hiba módszert az elkészített reprezentációra és találja meg a megoldást. 

In [85]:
# Válasz helye. Válaszát futtatható python kóddal adja meg.
print(trial_error(c).solution())

(4, 8, 0)
(12, 0, 0)
(6, 0, 6)
(0, 6, 6)
Got it
['12-->8', '8-->12', '12-->6', '12-->8']


6. (1 pont) Írja meg a szélességi keresést a megoldás megtalálásához. Írjon megjegyzés szabályos python formátumban amiben leírja, hogy pontosan mit csinál az algoritmus. Ha szükséges bármilyen egyéb program csomag a futtatáshoz annak megadásáról se feledkezzen meg.

In [86]:
# Válasz helye. Válaszát futtatható python kóddal adja meg.
from collections import deque

def breadth_first_tree_search(problem):
    #kezdő állapot kiovasása és FIFO sorba helyezése
    frontier = deque([Node(problem.initial)])

    #amig nem értük el a határt
    while frontier:
        #legészlsőbb elem kiemelése
        node = frontier.popleft()

        #ha cél állapotban vagyunk akkor vége
        if problem.goal_test(node.state):
            return node
        
        #a kiemelt elemből az összes új állapot legyártása az operátorok segítségével
        #valójában ez a bejárás
        frontier.extend(node.expand(problem))
        print(node.state)

7. (1 pont) Futtasa a szélességi keresést az elkészített reprezentációra és találja meg a megoldást. Ha szükséges bármilyen egyéb program csomag a futtatáshoz annak megadásáról se feledkezzen meg.

In [87]:
# Válasz helye. Válaszát futtatható python kóddal adja meg.
print(breadth_first_tree_search(c).solution())

(12, 0, 0)
(4, 8, 0)
(6, 0, 6)
(0, 8, 4)
(12, 0, 0)
(4, 2, 6)
['12-->6', '12-->8']
